In [1]:
import os
spark_version = 'spark-3.4.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 2s (109 kB/s)
Reading package lists... Done


In [23]:
#importing yfinance data
import yfinance as yf
celh = yf.download("CELH")
print(celh.head())

[*********************100%%**********************]  1 of 1 completed

                 Open       High        Low      Close  Adj Close  Volume
Date                                                                     
2007-01-22   8.000000  13.333333   8.000000  13.333333  13.333333   15075
2007-01-23  15.000000  15.000000  10.000000  10.600000  10.600000    9330
2007-01-24  10.666667  15.000000  10.666667  11.933333  11.933333   20415
2007-01-25  12.333333  13.400000  12.333333  13.333333  13.333333   12300
2007-01-26  12.800000  12.800000  12.666667  12.733333  12.733333   16275


In [22]:
#create a sparksession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("celh").getOrCreate()

In [24]:
#creating df in spark
pandas_df = celh
pandas_df.reset_index(drop=False,inplace=True)
df = spark.createDataFrame(pandas_df)
df.show(10)

+-------------------+------------------+------------------+------------------+------------------+------------------+------+
|               Date|              Open|              High|               Low|             Close|         Adj Close|Volume|
+-------------------+------------------+------------------+------------------+------------------+------------------+------+
|2007-01-22 00:00:00|               8.0|13.333333015441895|               8.0|13.333333015441895|13.333333015441895| 15075|
|2007-01-23 00:00:00|              15.0|              15.0|              10.0|10.600000381469727|10.600000381469727|  9330|
|2007-01-24 00:00:00|10.666666984558105|              15.0|10.666666984558105|11.933333396911621|11.933333396911621| 20415|
|2007-01-25 00:00:00|12.333333015441895|13.399999618530273|12.333333015441895|13.333333015441895|13.333333015441895| 12300|
|2007-01-26 00:00:00|12.800000190734863|12.800000190734863|12.666666984558105|12.733332633972168|12.733332633972168| 16275|
|2007-01

In [25]:
#creating temp view
df.createOrReplaceTempView('celh')

In [26]:
#query to find all dates between Jan 1st, 2021 and March 1st, 2024
all_data = spark.sql("""select * from celh where Date between "2021-01-01" and "2024-03-01" """).show(10)
all_data

+-------------------+------------------+------------------+------------------+------------------+------------------+--------+
|               Date|              Open|              High|               Low|             Close|         Adj Close|  Volume|
+-------------------+------------------+------------------+------------------+------------------+------------------+--------+
|2021-01-04 00:00:00| 17.55666732788086|17.573333740234375| 16.15999984741211| 16.51333236694336| 16.51333236694336| 5461500|
|2021-01-05 00:00:00|16.260000228881836| 18.59666633605957|15.716667175292969|17.383333206176758|17.383333206176758| 7765800|
|2021-01-06 00:00:00| 17.31999969482422|18.956666946411133| 17.22333335876465|17.446666717529297|17.446666717529297|21738600|
|2021-01-07 00:00:00|              17.5|18.816667556762695|16.803333282470703|18.780000686645508|18.780000686645508| 5086800|
|2021-01-08 00:00:00|18.670000076293945|19.586666107177734|18.663333892822266| 19.51333236694336| 19.51333236694336|11

In [14]:
#query to find only all closes between Jan 1st, 2021 and March 1st, 2024
closes = spark.sql("""select Close, Date from celh where Date between "2021-01-01" and "2024-03-01" """).show(10)
closes

+------------------+-------------------+
|             Close|               Date|
+------------------+-------------------+
| 16.51333236694336|2021-01-04 00:00:00|
|17.383333206176758|2021-01-05 00:00:00|
|17.446666717529297|2021-01-06 00:00:00|
|18.780000686645508|2021-01-07 00:00:00|
| 19.51333236694336|2021-01-08 00:00:00|
|19.036666870117188|2021-01-11 00:00:00|
|19.770000457763672|2021-01-12 00:00:00|
|19.866666793823242|2021-01-13 00:00:00|
|21.606666564941406|2021-01-14 00:00:00|
|             21.75|2021-01-15 00:00:00|
+------------------+-------------------+
only showing top 10 rows



In [20]:
#query to find only all adj closes between Jan 1st, 2021 and March 1st, 2024
adj_closes = spark.sql("""select `Adj Close`, Date from celh where Date between "2021-01-01" and "2024-03-01" """).show(10)
adj_closes

+------------------+-------------------+
|         Adj Close|               Date|
+------------------+-------------------+
| 16.51333236694336|2021-01-04 00:00:00|
|17.383333206176758|2021-01-05 00:00:00|
|17.446666717529297|2021-01-06 00:00:00|
|18.780000686645508|2021-01-07 00:00:00|
| 19.51333236694336|2021-01-08 00:00:00|
|19.036666870117188|2021-01-11 00:00:00|
|19.770000457763672|2021-01-12 00:00:00|
|19.866666793823242|2021-01-13 00:00:00|
|21.606666564941406|2021-01-14 00:00:00|
|             21.75|2021-01-15 00:00:00|
+------------------+-------------------+
only showing top 10 rows



In [29]:
#query for closes in descending order between the selected dates
biggest_closes = spark.sql("""select Close, Date from celh where Date between "2021-01-01" and "2024-03-01" order by Close desc""").show(10)
biggest_closes

+-----------------+-------------------+
|            Close|               Date|
+-----------------+-------------------+
|81.62000274658203|2024-02-29 00:00:00|
| 79.2699966430664|2024-03-01 00:00:00|
|68.41999816894531|2023-09-07 00:00:00|
| 67.7699966430664|2024-02-28 00:00:00|
| 67.5199966430664|2024-02-27 00:00:00|
| 67.3133316040039|2023-09-14 00:00:00|
|67.20999908447266|2023-09-08 00:00:00|
| 67.0433349609375|2023-09-12 00:00:00|
|66.85333251953125|2023-09-13 00:00:00|
|66.77999877929688|2023-09-06 00:00:00|
+-----------------+-------------------+
only showing top 10 rows

